**Sustainable Software Development, block course, March 2021**  
*Scientific Software Center, Institute for Scientific Computing, Dr. Inga Ulusoy*

# Analysis of the data

Imagine you perform a "measurement" of some type and obtain "scientific data". You know what your data represents, but you have only a vague idea how different features in the data are connected, and what information you can extract from the data.

You would start first with going through the data, making sure your data set is complete and that the result is reasonable. Imagine this already happened.

In the next step, you would inspect your data more closely and try to identify structures. That is the step that we are focusing on in this unit.

In the `data` folder, you will find several data files (`*.t` and `*.dat`). These are data files generated through some "new approach" that hasn't been used in your lab before. No previous analysis software exists, and you are going to establish a protocol for this "new approach" and "publish your results".

The data can be grouped into two categories: 
1. data to be analyzed using statistical methods;
2. data to be analyzed using numerical methods.

In your hypothetical lab, you are an "expert" in one particular "method", and your co-worker is an "expert" in the other. Combined these two methods will lead to much more impactful results than if only one of you analyzed the data. Now, the task in this course is to be solved collaboratively with your team member working on one of the analysis approaches, and you working on the other. You will both implement functionality into the same piece of "software", but do so collaboratively through git.

As you do not know yet which analysis is most meaningful for your data, and how to implement it, you will start with a jupyter notebook. You and your team member will work on the same notebook that will be part of a github repository for your project. This is the task for today. Discuss with your team members who will work on the statistical and who on the numerical analysis.

## Step 1

Generate a github repository with the relevant files.

## Step 2

Clone the repository to your local machine.

## Step 3

Start working on task 1 for your analysis approach. 

## Step 4

Create your own branch of the repository and commit your changes to your branch; push to the remote repository.

## Step 5

Open a `pull request` so your team member can review your implementation. Likewise, your team member will ask you to review theirs.

## Step 6

Merge the changes in your branch into `main`. Resolve conflicts.

## Step 7

Repeat working on task; committing and pushing to your previously generated branch or a new branch; open a pull request; merge with main; until you have finished all the tasks in your analysis approach. Delete obsolete branches.

# Start of the analysis notebook

**Author : Your Name**  
*Date : The date you started working on this*  
*Affiliation : The entity under whose name you are working on this*  

Place the required modules in the top, followed by required constants and global functions.

In [ ]:
# required modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb

In [ ]:
# constants and global functions
threshv = 1.0e-5
filenames = 'efield.t', 'expec.t', 'npop.t', 'nstate_i.t', 'table.dat'
filedir = '../../data/'

In [ ]:
# reading of the data files

# Statistical analysis

Find correlations in the data sets. Analyse the data statistically and plot your results.  

Here we would want to do everything with pandas and leave the data in a dataframe. The files that are relevant to you are `expect.t`, `npop.t` and `table.dat`.

### Task 1: Read in expec.t and plot relevant data

In [ ]:
# read and plot expec.t

# I/O module 

def readfile(filepath, mode, separator='\t', datatype=float, skiprows=1):
    """Reads a given (csv) file with the indicated separator.

        Args:
            filepath (str): path to the file including the filename
            mode (str):  indicates if data should be read into a Pandas dataframe ('df') or into a NumPy array ('npa')
            separator (str): optional, separator used in the file (e.g. '\t'), can also be a regular expression like '    |   |  '
            datatype (type): optional,
            skiprows (int): optional

        Returns:
            dataframe, array: A structured data object, either Pandas dataframe or NumPy array
    """
    if mode == 'df':
        data = pd.read_table(filepath, separator, engine='python')
    if mode == 'npa':
        data = np.loadtxt(filepath, dtype=datatype, skiprows=skiprows)
    return data


df_expect = readfile(filedir+filenames[1], 'df', '   ')



fig, ax = plt.subplots(2,figsize=(6,9))
fig.suptitle('Linear scale above, logarithmic scale below')


for i in range(len(df_expect.keys())):    
    ax[0].plot(df_expect[df_expect.keys()[i]], label=df_expect.keys()[i])

ax[0].legend()
ax[1].plot(df_expect)
ax[1].set_yscale('symlog')



We can discard the entries norm, \<x>, and \<y> as these are mostly constant.

In [ ]:
# eliminate columns based on the variance: if the variance of the values
# in a column is below a given threshold, that column is discarded

#print(df_expect.var())

def filterdata(data, treshhold=0, keeplist=[]):
    """Filters data based on variance. If variance is below a given threshold, column is discarded
        
        Arguments:
            data: data to be filtered, as dataframe
            treshhold: xx
            keeplist: optional, a list with column names to keep
    """

    for i in data.keys():
        if i in keeplist:
            continue
        if data[i].var() <= treshhold:
            print('Removing column: {}'.format(i))
            data = data.drop(i, axis=1)
    return data

        
print(filterdata(df_expect, threshv).var())

d = {'col1': [1, 2], 'col2': [3, 4]}

test_df = pd.DataFrame(data=d)


#print(filterdata(test_df, 0.4).var())
test1 = filterdata(test_df,0.9)

print(len(test1.columns))


### Task 2: Create plots of the relevant data and save as .pdf.

In [ ]:
# create plots

fig.savefig('../pdfPlots/expect.pdf',format='pdf')

### Task 3: Read in file `npop.t` and analyze correlations in the data

In [ ]:
 # read in npop.t

df_npop = readfile(filedir+filenames[2], 'df','    |   |  ')


In [ ]:
# discard all columns with variance below a set threshold - we can consider them as constant

df_npopfiltered = filterdata(df_npop, threshv, ['time'])
print(df_npopfiltered.var())

Plot the remaining columns. Seaborn prefers "long format" (one column for all measurement values, one column to indicate the type) as input, whereas the cvs is in "wide format" (one column per measurement type).

In [ ]:
# plot ideally with seaborn

npop_longFormat = pd.melt(df_npopfiltered, id_vars=['time'])

sb.lineplot(x='time', y='value', hue='variable', 
             data = npop_longFormat) 

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

## Quantify the pairwise correlation in the data

- negative correlation: y values decrease for increasing x - large values of one feature correspond to small values of the other feature
- weak or no correlation: no trend observable, association between two features is hardly observable
- positive correlation: y values increase for decreasing x - small values of one feature correspond to small values of the other feature

Remember that correlation does not indicate causation - the reason that two features are associated can lie in their dependence on same factors.

Correlate the value pairs using Pearson's $r$. Pearson's $r$ is a measure of the linear relationship between features:

$r = \frac{\sum_i(x_i − \bar{x})(y_i − \bar{y})}{\sqrt{\sum_i(x_i − \bar{x})^2 \sum_i(y_i − \bar{y})^2}}$

Here, $\bar{x}$ and $\bar{y}$ indicate mean values. $i$ runs over the whole data set. For a positive correlation, $r$ is positive, and negative for a negative correlation, with minimum and maximum values of -1 and 1, indicating a perfectly linear relationship. Weakly or not correlated features are characterized by $r$-values close to 0.

Other measures of correlation that can be used are Spearman's rank (value pairs follow monotonic function) or Kendall's $\tau$ (measures ordinal association), but they do not apply here. You can also define measures yourself.

In [ ]:
# print the correlation matrix

def correlatedata(data, corrmethod='pearson'):
    """Compute pairwise correlation of data with pandas.DataFrame.corr
    :param data: Data to be correlated.
    :type data: pandas dataframe (dict)
    :param corrmethod: Method to be used for correlation (pandas: pearson, kendall, spearman).
    :type corrmethod: string, optional
    """

    return data.corr(method=corrmethod)

df_npop_corr = correlatedata(df_npopfiltered, 'pearson')

print(df_npop_corr)

The diagonal values tell us that each value is perfectly correlated with itself. We are not interested in the diagonal values and also not in the correlation with time. We also need to get rid of redundant entries. Finally, we need to find the value pairs that exhibit the highest linear correlation. We still want to know if it is positive or negative correlation, so we cannot get rid of the sign.

In [ ]:
# get rid of time column, lower triangular and diagonal entries of the correlation matrix
# sort the remaing values according to their absolute value, but keep the sign

# drop time column
df_npop_corr = df_npop_corr.drop(['time'], axis=1)
#print(df_npop_corr)

# copied from solution for unit1
def get_correlation_measure(df):
    drop_values = set() # an unordered collection of items
    cols = df.columns # get the column labels
    print(cols)
    for i in range(0, df.shape[1]):
        for j in range(0, i+1): # get rid of all diagonal entries and the lower triangle
            drop_values.add((cols[i], cols[j]))
    print(drop_values)
    return drop_values

drop_vals = get_correlation_measure(df_npop_corr) # get rid of lower triangular and diagonal entries
corr2 = df_npop_corr.corr().unstack() # pivot the correlation matrix
corr2 = corr2.drop(labels=drop_vals).sort_values(ascending=False, key=lambda col: col.abs()) # sort by absolute values but keep sign
display(corr2)



Note that the entries in the left column are not repeated if they do not change from the row above (so the fourth feature pair is MO3 and MO6).

### Task 4: Print the resulting data to a file

In [ ]:
# write to file

def writefile(data, filepath, encoding='utf-8', header=False):
    """Writes a given dataframe to a csv file.
        Arguments:
        xxx
    """
    data.to_csv(filepath, encoding=encoding, header=header)
    
writefile(corr2, filedir+'npop_processed.csv')

#corr2.to_csv(filedir+'npop_processed.csv', encoding='utf-8', header=False)

### Task 5: Calculate the Euclidean distance (L2 norm) for the vectors in `table.dat`


The Euclidean distance measures the distance between to objects that are not points:

$d(p,q) = \sqrt{\left(p-q\right)^2}$

In this case, consider each of the columns in table.dat as a vector in Euclidean space, where column $r(x)$ and column $v(x)$ denote a pair of vectors that should be compared, as well as $r(y)$ and $v(y)$, and r(z) and v(z).

(Background: These are dipole moment components in different gauges, the length and velocity gauge.)

In [ ]:
# read in table.dat - I suggest reading it as a numpy array

#arr = np.loadtxt(filedir+filenames[4],dtype=float,skiprows=1)
arr = readfile(filedir+filenames[4],'npa')

# replace the NaNs by zero

#arr[np.where(np.isnan(arr))] = 0.

table_vals = np.nan_to_num(arr)

Now calculate how different the vectors in column 2 are from column 3, column 4 from column 5, and column 6 from column 7.

In [ ]:
# calculate the Euclidean distance



def euclidean_distance(list_ref, list_comp, vectors):
    distances = np.zeros(len(list_ref))
    for i in range(len(list_ref)):
        distances[i] = np.linalg.norm(vectors[list_comp[i]] - vectors[list_ref[i]])
    return distances



#dx = np.sqrt((arr[:,2]-arr[:,3])**2)
#dy = np.sqrt((arr[:,4]-arr[:,5])**2)
#dz = np.sqrt((arr[:,6]-arr[:,7])**2)

In [ ]:
# plot the result and save to a .pdf



out_dist = euclidean_distance([2,4,6],[3,5,7],table_vals)
print(out_dist)
x = range(0,len(out_dist))
plt.bar(x,out_dist)
plt.xticks(x, ('x', 'y', 'z'))
plt.show()




#plt.plot(dx,label="column 2,3")
#plt.plot(dy,label="column 4,5")
#plt.plot(dz,label="column 6,7")
#plt.xlabel("table index")
#plt.ylabel("euclidean distance")
#plt.legend()

In [ ]:
# print the result to a file

plt.savefig("euclideandistance.pdf")

# Numerical analysis

Analyze the data using autocorrelation functions and discrete Fourier transforms. Plot your results.

In [ ]:
# define some global functions

### Task 1: Read in `efield.t` and Fourier-transform relevant columns

In [ ]:
# read and plot efield.t

efield = np.loadtxt(filedir+filenames[0],dtype=float,skiprows=1)

plt.plot(efield[:,0],efield[:,1],label="efield x")
plt.plot(efield[:,0],efield[:,2],label="efield y")
plt.plot(efield[:,0],efield[:,3],label="efield z")

plt.xlabel("time")
plt.ylabel("field value")
plt.legend()

Here we are interested in column 2 since the others are constant.

In [ ]:
# discard the columns with variance below threshold - these are considered constant

print(np.var(efield[:,1]))
print(np.var(efield[:,2]))
print(np.var(efield[:,3]))

efield = np.delete(efield,[1,3],1)

In [ ]:
# discrete Fourier transform of the remaining column: You only need the real frequencies

efield_fft = np.fft.fft(efield[:,1])

freq = np.fft.fftfreq(efield.shape[0],d=(efield[1,0]-efield[0,0]))

### Task 2: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot your results

plt.plot(freq,efield_fft.real,label="fft real part")
plt.xlabel("frequency f")
plt.ylabel("P(f)")
plt.legend()
plt.savefig('efield_fft.pdf')

### Task 3: Calculate the autocorrelation function from nstate_i.t
The autocorrelation function measures how correlated subsequent vectors are with an initial vector; ie. 

$\Psi_{corr} = \langle \Psi(t=0) | \Psi(t) \rangle = \int_0^{tfin} \Psi(0)^* \Psi(t) dt$

Since we are in a numerical representation, the integral can be replaced with a sum; and the given vectors are already normalized.

In [ ]:
# read in as numpy array
nstate_i = np.loadtxt(filedir+filenames[3],dtype=float,skiprows=1)

In [ ]:
# store the time column (column 0) in a vector and drop from array

def get_time_and_update_data(data):
    
    time = data[:, 0]
    new_data = np.delete(data, 0, 1)
    
    return time, new_data

time, update_nstate_i = get_time_and_update_data(nstate_i)

In [ ]:
# correct the data representation: this is in fact a complex matrix
# the real part of each matrix column is contained in numpy array column 0, 2, 4, 6, ...
# the imaginary part of each matrix column is contained in numpy array column 1, 3, 5, 7, ...
# convert the array that was read as dtype=float into a dtype=complex array
def get_complex_array(array):
    
    complex_array = array[:, 0::2] + 1j * array[:, 1::2]
    
    return complex_array

update_nstate_i = get_complex_array(update_nstate_i)

In [ ]:
# for the autocorrelation function, we want the overlap between the first vector at time 0 and all 
# subsequent vectors at later times - the sum of the product of initial and subsequent vectors for each time step

def get_autocorrelation(array):
    
    auto_corr_matrix = []
    for i in range(1, len(array)):
        corr = np.multiply(array[0, :], array[i, :])
        auto_corr_matrix.append(np.array(corr))
    auto_correlation = np.sum(auto_corr_matrix, axis=1)
    
    return auto_correlation

auto_correlation = get_autocorrelation(update_nstate_i)


### Task 4: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the autocorrelation function - real, imaginary and absolute part
plt.subplot(1, 3, 1)
plt.gca().set_title('Real part')
plt.plot(time[1:], auto_correlation.real)


plt.subplot(1, 3, 2)
plt.gca().set_title('Imag part')
plt.plot(time[1:], auto_correlation.imag)

plt.subplot(1, 3, 3)
plt.gca().set_title('Abs part')
plt.plot(time[1:], np.abs(auto_correlation))

plt.show()

### Task 5: Discrete Fourier transform of the autocorrelation function

In [ ]:
# discrete Fourier-transform the autocorrelation function - now we need all frequency components, 
# also the negative ones

fourier = np.fft.fft(auto_correlation)

### Task 6: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the power spectrum (abs**2)

def get_power_spectrum(fourier):
    
    return abs(fourier)**2


plt.plot(get_power_spectrum(fourier))
plt.title('Power spectrum')
plt.show()